# Exploring the data

### Working with tabular data

In [85]:
import arcpy
import os
import pandas as pd

# Read the data and store it in a pandas dataframe, df.  
# A data frame is a 2-dimensional data structure.  
projectDir = r'C:\Users\echoi4\Desktop\esterArcGISProject-20230428T013544Z-001\esterArcGISProject' ## Replace with your data directory
try:
    # When working inside ArcGIS Pro,
    # use the project name "CURRENT"
    aprx = arcpy.mp.ArcGISProject("CURRENT")
except:
    # When working outside of ArcGIS Pro,
    # use the project full path file name.
    aprx = arcpy.mp.ArcGISProject(project_dir + "/estherArcGISProject.aprx")
    
workingDir = projectDir + "\outputFreq\\"
fileGDB = projectDir + '\esterArcGISProject.gdb\\'
visDir = workingDir + '\picsFreq\\'

# Create output directories if they don't already exist.
dirs = [workingDir, visDir]
for d in dirs:
    if not os.path.exists(d):
        os.mkdir(d)

myData = projectDir + '\essie2.csv' #'tuta_absoluta.csv' 
df = pd.read_csv(myData)

# Print the first 5 records to get a feel for the data.
df.head()

[85]:28: DtypeWarning: Columns (0,16,20,22,24,25,26,27,28,29,34,35,36,37,38,39,40,41,43,44,46,47,51,52,55,56,57,61,62,63,64,65,66,67,68,71,72,73,75,76,77,79,80,81,82,83,84,85,86,88,89,92,93,94,95,96) have mixed types. Specify dtype option on import or set low_memory=False.


,Report:,Bulk Mentions Download,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98
0,Brand:,AS - Plant pests - 12/9/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,From:,Wed Jan 01 00:00:00 UTC 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,To:,Sun Jan 02 00:00:00 UTC 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Label:,3 yr Tuta Abs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# What fields does the data have?
df.columns

Index(['Number', 'Page Type', 'Country', 'Year', 'Full Text'], dtype='object')

In [36]:
# How many columns and rows does the data have?
df.shape

(490, 5)

In [37]:
# Which countries are there in the "Country" field?
# The Series method named unique returns a Numpy array containing the unique values in a series. 
df['Country'].unique()

array(['United States', 'Australia', 'Canada', 'Netherlands',
       'United Kingdom', 'Poland', 'Japan', 'New Zealand', 'Italy',
       'Vietnam', 'Republic of Ireland', 'Finland', 'Croatia', 'Israel',
       'Germany', 'The Bahamas', 'Brazil', 'Russia', 'Norway',
       'Indonesia', 'South Africa', 'Pakistan', 'India',
       'Republic of Serbia', 'Argentina', 'Singapore', 'Sweden',
       'Colombia', 'Mexico', 'Gibraltar', 'Venezuela'], dtype=object)

In [38]:
# Change the name in the Country column for USA to match GIS data name for USA
df['Country'] = df['Country'].replace('United States of America','United States')
df['Country'].unique()

array(['United States', 'Australia', 'Canada', 'Netherlands',
       'United Kingdom', 'Poland', 'Japan', 'New Zealand', 'Italy',
       'Vietnam', 'Republic of Ireland', 'Finland', 'Croatia', 'Israel',
       'Germany', 'The Bahamas', 'Brazil', 'Russia', 'Norway',
       'Indonesia', 'South Africa', 'Pakistan', 'India',
       'Republic of Serbia', 'Argentina', 'Singapore', 'Sweden',
       'Colombia', 'Mexico', 'Gibraltar', 'Venezuela'], dtype=object)

In [39]:
# How many countries is that?
# Use the built-in len method to find the length of the array of unique country names.
print(len(df['Country'].unique()))

31


### Aggregating the posts to get a weight for each country
We would like to visualize how many posts are appearing in each country each year.  This means we need to get a frequency count by country for each year.
We'll use Pandas (although this can also be done with arcpy by calling TabletoTable to make a DBF table and then calling the Frequency tool.)

##### Get a frequency table with Pandas

The dataframe method named value_counts can be used to get the frequency of the posts in a country each year.  This returns a pandas series.

In [40]:
freq_series = df.value_counts(["Year", "Country"])
freq_df = freq_series.to_frame()
freq_df

0
Year Country          
2017 United States  45
2014 United States  43
2018 United States  38
2016 United States  36
2017 Australia      33
...                 ..
     Russia          1
     South Africa    1
2018 Australia       1
     Croatia         1
2022 United States   1

[75 rows x 1 columns]

See the dimensions in the above output?  This makes each year + country pairings the index for each row (the unique identifier field).   And the frequncy value is the **one column** .  The following code lists the index values (year-country tuples):

In [41]:
print(freq_df.index.values)
print(freq_df.columns)

[(2017, 'United States') (2014, 'United States') (2018, 'United States')
 (2016, 'United States') (2017, 'Australia') (2015, 'United States')
 (2016, 'Australia') (2019, 'United States') (2018, 'Canada')
 (2015, 'Canada') (2019, 'Canada') (2020, 'United States')
 (2016, 'United Kingdom') (2020, 'Canada') (2021, 'Canada')
 (2016, 'Canada') (2021, 'United States') (2017, 'United Kingdom')
 (2016, 'India') (2014, 'United Kingdom') (2017, 'Canada')
 (2015, 'United Kingdom') (2014, 'Canada') (2019, 'Japan') (2015, 'Italy')
 (2020, 'Australia') (2015, 'Sweden') (2018, 'United Kingdom')
 (2013, 'United States') (2019, 'United Kingdom') (2014, 'Colombia')
 (2019, 'Italy') (2019, 'Australia') (2019, 'Finland')
 (2018, 'The Bahamas') (2013, 'Australia') (2019, 'Republic of Ireland')
 (2019, 'Vietnam') (2018, 'Japan') (2020, 'Japan') (2020, 'New Zealand')
 (2020, 'Poland') (2020, 'United Kingdom') (2021, 'Australia')
 (2021, 'Netherlands') (2021, 'United Kingdom')
 (2018, 'Republic of Ireland') (

Following this approach described on [stackoverflow]{https://stackoverflow.com/questions/47136436/python-pandas-convert-value-counts-output-to-dataframe}, we'll reset the index.  This generates a new unique ID (index) for each row.   Then we can rename the columns, to name our new column "Frequency".

In [42]:
freq_df_reset = freq_df.reset_index()
freq_df_reset
# Rename the last column to "Frequency"
freq_df_reset.columns = ["Year", "Country","Frequency"]
## Add code to print the head of freq_df_reset

In [43]:
# Get the overall maximum and minimum frequencies for later use.
max_country_count = freq_df_reset['Frequency'].max()
min_country_count = freq_df_reset['Frequency'].min()
print(max_country_count)
print(min_country_count)

## Add code to print the maximum and minimum year in the data.


45
1


### Make a copy of the worldwide countries dataset

Other data will be joined to this, so we want to preserve the original before modifying the table.

Data source: 
[ArcGIS Hub World Countries Generalized] (https://hub.arcgis.com/datasets/2b93b06dc0dc4e809d3c8db5cb96ba69_0/explore?location=-0.101905%2C0.000000%2C2.09)

In [44]:
import arcpy
arcpy.env.workspace = projectDir + r'esterArcGISProject.gdb'
arcpy.env.overwriteOutput = True
worldData = 'world_countries'
worldDataCopy = workingDir + 'worldEstherFreq.shp'
arcpy.CopyFeatures_management(worldData, worldDataCopy)

<Result 'C:\\Users\\echoi4\\Desktop\\esterArcGISProject-20230428T013544Z-001\\esterArcGISProject\\outputFreq\\worldEstherFreq.shp'>

In [45]:
# Get a the base name (without the extention) of the input data.  We'll use this to name files in the next part.
baseName = os.path.basename(myData)
baseName = os.path.splitext(baseName)[0]
baseName = baseName.split("_")[0]
baseName

'essie'

### Create a table for each time step and join the tables with the GIS data

In [46]:
import traceback
# Create an empty dictionary to store the number of records per year.
yearly_count = {}
max_country_count = freq_df_reset['Frequency'].max()
min_country_count = freq_df_reset['Frequency'].min()
# For each year in the reduced dataset, create a csv file.

for yr in sorted(freq_df_reset['Year'].unique()):
    # Select rows based on the year
    rslt_df = freq_df_reset[freq_df_reset['Year'] == yr]
    recordsThisYear = rslt_df['Frequency'].sum() 
    print(f"Number of records in {yr}: {recordsThisYear}" ) 
    yearly_count[yr] = len(rslt_df)

    # Prepare an output name
    annualCSV = workingDir + f'{baseName}{yr}.csv'
    
    # Write the data frame to csv
    rslt_df.to_csv(annualCSV)
    
    print(f'Year:   {yr}')
    print(rslt_df['Country'].unique())
    
    # Join the data
    joinedData = arcpy.management.AddJoin(
        in_layer_or_view = worldDataCopy, 
        in_field = 'COUNTRY', 
        join_table = annualCSV, 
        join_field = 'Country', 
    join_type = 'KEEP_COMMON')
    
    curData = fileGDB + f'{baseName}{yr}'
    try:
        # The frequency field automatic name is quite long.  E.g., esther_reduced2013_csv_Frequency
        # Let's rename it.
        
        cur_frequency_field = f'{os.path.basename(annualCSV).replace(".","_")}_Frequency'
        frequency_field = 'Count'
        arcpy.CopyFeatures_management(joinedData, curData)
        fds = arcpy.ListFields(curData)
        print(f"Field names before: {[f.name for f in fds]}")
        arcpy.management.AlterField(curData, cur_frequency_field, frequency_field, frequency_field)
        fds = arcpy.ListFields(curData)
        print(f"Field names after: {[f.name for f in fds]}")
        shpData = workingDir + f'{baseName}{yr}.shp'
        arcpy.management.CopyFeatures(curData, shpData)
        
    except:
        traceback.print_exc()
        print("File already exists.  New copy not created.")

Number of records in 2013: 3
Year:   2013
['United States' 'Australia']
Field names before: ['OBJECTID', 'Shape', 'worldEstherFreq_FID_1', 'worldEstherFreq_COUNTRY', 'worldEstherFreq_ISO', 'worldEstherFreq_COUNTRYAFF', 'worldEstherFreq_AFF_ISO', 'worldEstherFreq_SHAPE_Leng', 'worldEstherFreq_Shape_Le_1', 'essie2013_csv_Field1', 'essie2013_csv_Year', 'essie2013_csv_Country', 'essie2013_csv_Frequency', 'Shape_Length', 'Shape_Area']
Field names after: ['OBJECTID', 'Shape', 'worldEstherFreq_FID_1', 'worldEstherFreq_COUNTRY', 'worldEstherFreq_ISO', 'worldEstherFreq_COUNTRYAFF', 'worldEstherFreq_AFF_ISO', 'worldEstherFreq_SHAPE_Leng', 'worldEstherFreq_Shape_Le_1', 'essie2013_csv_Field1', 'essie2013_csv_Year', 'essie2013_csv_Country', 'Count', 'Shape_Length', 'Shape_Area']
Number of records in 2014: 60
Year:   2014
['United States' 'United Kingdom' 'Canada' 'Colombia' 'Gibraltar' 'India'
 'Mexico' 'Sweden' 'Venezuela']
Field names before: ['OBJECTID', 'Shape', 'worldEstherFreq_FID_1', 'worldE

Field names after: ['OBJECTID', 'Shape', 'worldEstherFreq_FID_1', 'worldEstherFreq_COUNTRY', 'worldEstherFreq_ISO', 'worldEstherFreq_COUNTRYAFF', 'worldEstherFreq_AFF_ISO', 'worldEstherFreq_SHAPE_Leng', 'worldEstherFreq_Shape_Le_1', 'essie2022_csv_Field1', 'essie2022_csv_Year', 'essie2022_csv_Country', 'Count', 'Shape_Length', 'Shape_Area']


#### Add records so that graduated colors will be normalized

If we use graduated colors for the count on a year that has min 1 and max 5, the 5 will be colored the same as the max in another year that has max of 100.  As a way of normalizing the data for the purposes of visualizing it, we'll add a two new dummy records to each dataset. One new record will have the maximum value of all the years in the Count column.  The other will have the overall minimum in the Count column. The records will be dummies in the sense that they won't have any geometry, so they won't be displayed on the map, but they will still normalize the graduated colors so that the color steps are consistant across the years.  

In [47]:
for yr in sorted(freq_df_reset['Year'].unique()):
    try:
        curData = fileGDB + f'{baseName}{yr}'
        with arcpy.da.InsertCursor(curData, [frequency_field]) as ic:
            newRecord = [max_country_count]
            ic.insertRow(newRecord)
            newRecord = [min_country_count]
            ic.insertRow(newRecord)
        del ic
    except:
        print(arcpy.GetMessages() )

##### Color ramps
We're going to use graduated color for our frequency counts.  You can specify color ramps by name.  
See a complete list of color ramps by uncommenting the following code (remove the hash tags at the beginning of each line)

In [49]:
#aprx = arcpy.mp.ArcGISProject(projectDir + 'estherArcGISProject.aprx')
#ramps = aprx.listColorRamps()
#for ramp in ramps:
    print (ramp.name)

OSError: C:\Users\echoi4\Desktop\esterArcGISProject-20230428T013544Z-001\esterArcGISProjectestherArcGISProject.aprx

In [50]:
# aprx is the ArcGIS project object

# Get a list of maps in this project.
myMap = aprx.listMaps('Map')[0]
layers = myMap.listLayers()
layer_object = layers[0]
count = 0 
while "Worldwide" not in layer_object.name and count < len(layers):
    print(layer_object.name)
    myMap.removeLayer(layer_object)
    count = count + 1
    layer_object = layers[count]
    
for layer in layers:
    print(layer.name)

Worldwide Countries
Human Geography Dark Label
Human Geography Dark Detail
Human Geography Dark Base


### Create map layers and take screenshots

We'll do this by looping through the years. Earlier, we created a shapefile for each year.  Now we'll make a layer for each shapefile.   We'll add it to the map, capture a screenshot (a png image), and turn then that layer's visibility off, so that we can do the same again with the next shapefile.

In [51]:
# Get a list of maps in this project.
myMap = aprx.listMaps('Map')[0]

custom_layout = aprx.listLayouts("*surround*")[0]
print(f"Layout elements on {custom_layout.name}:")
for elem in custom_layout.listElements():
    print("\t" + elem.name)
    if "Text" in elem.name:
        textbox = elem


RGBs = [[190, 210, 255, 60], [245, 122, 122, 60], [255, 255, 0, 60]]

# For each year in the data, add a layer and export an image of it.
for yr in sorted(freq_df_reset['Year'].unique()):

    # Get the name of the shapefile associated with the current year. 
    curData = workingDir + f'{baseName}{yr}.shp'
    
    # Add the shapefile as a layer on the map.
    myMap.addDataFromPath(curData)
    # Set the text string to current year
    textbox.text = yr
    
    # Now we need the layer object associated with our new layer. 
    # This gives access to the symbology properties of the layer.  
    # Get a list of layor objects for the layers on this map.
    layerObjects = myMap.listLayers()
    
    # Get the first one (the most recently added to the map)
    lay = layerObjects[0]

    print(f'Creating the {lay.name} layer.')
    
    sym = lay.symbology

    if lay.isFeatureLayer and hasattr(sym, "renderer"):
    
        # Modify the symbology object.
        sym.updateRenderer("GraduatedColorsRenderer")
        sym.renderer.classificationField = frequency_field
        sym.renderer.classificationMethod = "EqualInterval"
        ## Modify the number of breaks and the color ramp (see the code box before this for choices).
        sym.renderer.breakCount = 5
        sym.renderer.colorRamp = aprx.listColorRamps("Yellow-Orange-Red (5 Classes)")[0]

        # Set the layer's symbology to the updated symbology object.
        lay.symbology = sym 

    lay.symbology = sym
   
    # Capture a screen shot of the newly added and symbolized layer.
    outPic = visDir+f'{baseName}{str(yr).zfill(2)}.png'
    custom_layout.exportToPNG(outPic)
    # Turn the layer off to prepare for the adding the next one and capturing an image.
    lay.visible = False
    
    del lay

# Capture an image of the legend.
# Turn on one of the layers so that the legend will show the patches
lay = layerObjects[0]


lay.visible = True

# The project has a layout specifically designed to display only the legend.
# Here we're getting the layout object associated with it.  
# This layout view has the word 'legend' in it.
custom_layout = aprx.listLayouts("*legend*")[0]

# Enlarge the legend patches.  The default is too small for our purposes.
leg = custom_layout.listElements('LEGEND_ELEMENT')[0]

for itm in leg.items:
    # Specify the size in pts
    itm.patchHeight = 50
    itm.patchWidth = 100

legendName = visDir + f'legend.png'
custom_layout.exportToPNG(legendName)
    
del myMap
del aprx


Layout elements on Layout_w_surrounds:
	Legend
	Text
	North Arrow
	Map Frame
Creating the essie2013 layer.
Creating the essie2014 layer.
Creating the essie2015 layer.
Creating the essie2016 layer.
Creating the essie2017 layer.
Creating the essie2018 layer.
Creating the essie2019 layer.
Creating the essie2020 layer.
Creating the essie2021 layer.
Creating the essie2022 layer.


In [52]:
curData
x = "C:/gispy/esterArcGISProject/esterArcGISProject.gdb/essie2013"
#myMap.addDataFromPath(x)
arcpy.Exists(x)

False

For more graduated color arcpy examples, see this 'how to' page: [https://support.esri.com/en/technical-article/000020468]

For more information about legend items, see this ESRI help page: [https://pro.arcgis.com/en/pro-app/2.8/arcpy/mapping/legenditem-class.htm]

### Creat an animation

In [53]:
import imageio
images = []
import os
mapPics = os.listdir(visDir)

movieName = visDir + '/movie.gif'
if os.path.exists(movieName):
    os.remove(movieName)

for pic in mapPics:
    if pic.endswith("png") and "legend" not in pic:
        images.append(imageio.imread(visDir + pic))


imageio.mimsave(movieName, images, duration=0.8)
image_count = len(images)
print('{} created using {} maps!'.format( movieName, image_count))


[53]:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


C:\Users\echoi4\Desktop\esterArcGISProject-20230428T013544Z-001\esterArcGISProject\outputFreq\\picsFreq\/movie.gif created using 10 maps!


In [54]:
# Check file names.
print(mapPics)

['essie2013.png', 'essie2014.png', 'essie2015.png', 'essie2016.png', 'essie2017.png', 'essie2018.png', 'essie2019.png', 'essie2020.png', 'essie2021.png', 'essie2022.png', 'legend.png']


### Create an html display

In [55]:
header = """<!DOCTYPE html>
<html>
<head>
<style>
div.gallery {
  border: 1px solid #ccc;
}

div.gallery:hover {
  border: 1px solid #777;
}

div.gallery img {
  width: 100%;
  height: auto;
}

div.desc {
  padding: 15px;
  text-align: center;
}

* {
  box-sizing: border-box;
}

.responsive {
  padding: 0 6px;
  float: left;
  width: 24.99999%;
}

@media only screen and (max-width: 700px) {
  .responsive {
    width: 49.99999%;
    margin: 6px 0;
  }
}

@media only screen and (max-width: 500px) {
  .responsive {
    width: 100%;
  }
}

.clearfix:after {
  content: "";
  display: table;
  clear: both;
}
</style>
</head>
<body>
""" 

bodyContent = f"""<h2>{baseName.title()} Posts Over {image_count} Years </h2>
<h4>Author: Jane Doe</h4>
<h4>Data source: {baseName} social media posts</h4>"""

mapPics = os.listdir(visDir)

mapPics = [picName for picName in mapPics if picName.endswith('.png') and "legend" not in picName ]

for pic in mapPics:
    dateVal = os.path.splitext(pic)[0]
    dateVal = dateVal.replace(baseName,'')
    dateVal = int(dateVal)

    bodyContent = bodyContent + f"""<div class="responsive"><div class="gallery">
  <a target="_blank" href={pic}>
    <img src={pic} alt="{os.path.splitext(pic)[0]}" width="800" height="600">
  </a>
  <div class="desc">{yearly_count[dateVal]} {baseName.title()} post(s) in {dateVal}</div>
</div></div>"""

# Add legend
legendName = os.path.basename(legendName)
bodyContent = bodyContent + f"""<div class="responsive"><div class="gallery">
  <a target="_blank" href={legendName}>
    <img src={legendName} alt="animation" width="800" height="600">
  </a>
  <div class="desc">{baseName.title()} post count ranges</div>
</div></div>"""

# Add movie
movieName = os.path.basename(movieName)
bodyContent = bodyContent + f"""<div class="responsive"><div class="gallery">
  <a target="_blank" href={movieName}>
    <img src={movieName} alt="animation" width="800" height="600">
  </a>
  <div class="desc">{baseName.title()} posts over time.</div>
</div></div>"""


footer = """</body>
</html>"""

html_file_name = visDir + f'{baseName}.html'

with open( html_file_name,'w') as outFileObject:
  outFileObject.write(header)
  outFileObject.write(bodyContent)
  outFileObject.write(footer)
os.startfile(html_file_name)